In [41]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

{'name': 'e4e1b4447e20',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'DNoCuE3RTv2p_fQWsmn1YA',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [42]:
path = '/opt/ml/data/wikipedia_documents.json'

In [43]:
import json #json 수정하려고 열음
with open(path, 'r') as f:
    wiki = json.load(f)

In [44]:
for i in range(len(wiki)):
    del wiki[str(i)]['corpus_source']# 필요없는거 삭제
    del wiki[str(i)]['domain']
    del wiki[str(i)]['author']
    del wiki[str(i)]['html']
    del wiki[str(i)]['document_id']
    del wiki[str(i)]['url']

In [45]:
wiki['0']['text']

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [46]:
import re
for i in range(len(wiki)):
    temp = wiki[f'{i}']['text']
    temp = re.sub('[#*~"\"]','',temp)
    temp = temp.split('\n')
    temp = [s.strip() for s in temp]
    temp = list(filter(None, temp))
    temp = ' '.join(temp)
    wiki[f'{i}']['text'] = temp

In [47]:
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 국가 문서에서 설명하고 있다.',
 'title': '나라 목록'}

In [48]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type" : "stop",
                        "stopwords_path" : "my_stopwords.txt" # /etc/elastic안에 txt파일이 존재해야 댑니다
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "mixed",
                        "filter" : ["my_stop_filter"]# 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", # 먼지 잘모르겟
            "properties": {
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "title":{"type": "text"}
                }
            }
        }

In [49]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

<ipython-input-49-c1b8263f83d6>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=index_config)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [50]:
import time
import tqdm

for doc_id, doc in tqdm.tqdm(wiki.items(),total=len(wiki)):
    
    es.index(index=INDEX_NAME, id=doc_id, body=doc)
    # time.sleep(0.1) # 문서집어넣기 time슬립너어야 에러 안걸림 (너무빨리 문서를 호출해서)

  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-50-5a9fb178b7ff>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, id=doc_id, body=doc)
100%|██████████| 60613/60613 [05:25<00:00, 186.24it/s]


In [51]:
import pprint
# doc = es.get(index=INDEX_NAME, id=60000)#
# pprint.pprint(doc)

In [52]:
tv = es.termvectors(index=INDEX_NAME, id=2, body={"fields" : ["content","title"]})

In [53]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 5747,
                                                 'sum_doc_freq': 9373,
                                                 'sum_ttf': 9378},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 0}


In [54]:
from datasets import load_dataset

In [55]:
import datasets
import pandas as pd
import re
import pandas as pd
from datasets import load_from_disk
trainpath = "/opt/ml/data/train_dataset"
testpath = '/opt/ml/data/test_dataset'
datasets = load_from_disk(testpath)
datasets

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})

In [56]:
query = datasets['validation']['question']
# ground_truth = datasets['validation']['context']
id = datasets['validation']['id']
len(query)

600

In [57]:
id[0]

'mrc-1-000653'

In [58]:
correct = 0
total = []
for i, q in enumerate(query):
    q = q.replace('~','-')
    res = es.search(index=INDEX_NAME, q=q, size=3)
    # print('query : ', q)
    # print('id : ', id[i])
    # print(res)
    tmp = {
        'id':id[i],
        'text1':res['hits']['hits'][0]['_source']['text'],
        'text2':res['hits']['hits'][1]['_source']['text'],
        'text3':res['hits']['hits'][2]['_source']['text'],
    }
    total.append(tmp)
    
    # context = res['hits']['hits']
    # print(q)
    # print(context)
    # print('='*100)


# print('% : ', correct/len(query))

In [59]:
import pandas as pd
df = pd.DataFrame(total)
df


,id,text1,text2,text3
0,mrc-1-000653,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,글리제 581 g는 천칭자리 방향으로 지구로부터 약 20.5광년 떨어진 적색 왜성 ...,"지구 근접 소행성(地球近接小行星)은 지구 궤도와 교차하는 궤도를 가진 소행성으로, ..."
1,mrc-1-001113,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...,SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌...,2014년 1월 10일 KBO 리그 LG 트윈스와 계약하며 2013년 이후 방출된 ...
2,mrc-0-002191,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ...",돌푸스는 니더외스터라이히주의 텍싱탈에서 싱글맘이자 신실한 가톨릭교인인 어머니 밑에서...,; 디오의 성 : 1부의 무대. 흡혈귀 디오가 근거지로 하고 있던 곳이다. 배경에는...
3,mrc-0-003951,"유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민...",멕시코의 언어는 다양하고 그 중 스페인어가 가장 널리 쓰인다. 멕시코 원주민 언어들...,"농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리포르니아주 북부에서는..."
4,mrc-1-001272,1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 파업을 지지했다는...,제1차 인도차이나 전쟁 중에는 프랑스 공군의 깟비 공군기지로 이용되었다. 1954년...,그는 1940년대에 자신의 군사 경력을 시작하여 프랑스 식민지 육군에 입대하였다. ...
...,...,...,...,...
595,mrc-0-002989,"재단 X가 슈라우드(=소노자키 후미네)가 개발한 가이아 메모리의 테크놀로지를 분석,...","후토 타워의 완공 30주년이 되는 해의 여름, A에서 Z까지의 26개의 차세대 가이...",NEVER(NEVER(ネバー)|네바)는 다이도 카츠미가 이끄는 세계적으로 악명 높은...
596,mrc-0-001804,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...,과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망한 사망자의...,4.19 의거나 3.15 부정선거 당시 학생 시위 진압자에 대한 미온적인 처벌이 논...
597,mrc-0-003411,"2005년(제56회 / 지바 롯데 마린스 대 한신 타이거스 1차전·10월 22일, ...",콜드게임(called game)이란 야구나 소프트볼 등의 운동경기에서 심판에 의하여...,《디스아너드 2》(Dishonored 2)는 아케인 스튜디오가 개발하고 베데스다 소...
598,mrc-0-003436,주노 해변\n\n주노 해변은 쿠르쇨르메르의 양쪽으로 뻗은 5마일 정도의 해변이었다....,제3사단은 치명적인 운명을 맞은 영국 해외원정군의 일부였다. 이들은 제2차 세계 대...,캐나다군은 독일군의 베리에르 능선 방어가 여전히 철저하다는 것을 알고 있었다.D'E...


In [60]:
df.to_csv('../add_context_test_dataset.csv')